In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('twitter_training.csv',encoding='ISO-8859-1')
df

,Id,Entity,Sentiment,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [3]:
df.drop(['Id','Entity'],axis=1,inplace=True)
df.head()

,Sentiment,Text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  74682 non-null  object
 1   Text       73824 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [5]:
df.duplicated().sum()

5254

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.duplicated().sum()

0

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df.isnull().sum()

Sentiment    0
Text         4
dtype: int64

In [10]:
df['Text']=df['Text'].fillna(df['Text'].mode()[0])

In [11]:
df.isnull().sum()

Sentiment    0
Text         0
dtype: int64

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
corpus = []
for i in range(0, len(df)):
    # Remove URLs
    word = re.sub(r'http\S+|www\S+|https\S+',' ', df['Text'][i], flags=re.MULTILINE)
    # Remove email id
    word = re.sub(r'\S+@\S+',' ',word)
    word = re.sub(r'[^\w\s]',' ',word)  # This will remove punctuation and emojis
    word = re.sub(r'\d+',' ',word)  # Remove numbers
    word = word.lower()
    word = word.split()
    word = [lemmatizer.lemmatize(j) for j in word if j not in set(stopwords.words('english'))]
    word = ' '.join(word)
    corpus.append(word)

corpus

['im getting borderland murder',
 'coming border kill',
 'im getting borderland kill',
 'im coming borderland murder',
 'im getting borderland murder',
 'im getting borderland murder',
 'spent hour making something fun know huge borderland fan maya one favorite character decided make wallpaper pc original image versus creation made enjoy pic twitter com mlsi wf jg',
 'spent couple hour something fun know huge borderland fan maya one favorite character decided make wallpaper pc original picture compared creation made fun pic twitter com mlsi wf jg',
 'spent hour something fun know huge borderland fan maya one favorite character',
 'spent hour making something fun know huge rhandlerr fan maya one favorite character decided make wallpaper pc original image versus creation made enjoy pic twitter com mlsi wf jg',
 'spent hour making something fun know huge rhandlerr fan maya one favorite character decided make wallpaper pc original image versus creation made enjoy pic twitter com mlsi wf jg

## One method

In [16]:
# Create Bag Of Words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,binary=True,ngram_range=(1, 2))
x=cv.fit_transform(corpus).toarray()
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
df.Sentiment.value_counts()

Sentiment
Negative      21164
Positive      19024
Neutral       17008
Irrelevant    12232
Name: count, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder
df.Sentiment=LabelEncoder().fit_transform(df.Sentiment)

In [19]:
y=df.Sentiment
y

0        3
1        3
2        3
3        3
4        3
        ..
69423    3
69424    3
69425    3
69426    3
69427    3
Name: Sentiment, Length: 69428, dtype: int32

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(52071, 2500)
(17357, 2500)
(52071,)
(17357,)


In [21]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
from sklearn.metrics import *
print(f'accuracy_score:{accuracy_score(y_test,y_pred)}')
print(f'classification_report:{classification_report(y_test,y_pred)}')

accuracy_score:0.8821224866048281
classification_report:              precision    recall  f1-score   support

           0       0.90      0.83      0.86      3125
           1       0.90      0.91      0.91      5378
           2       0.90      0.87      0.89      4149
           3       0.83      0.90      0.87      4705

    accuracy                           0.88     17357
   macro avg       0.88      0.88      0.88     17357
weighted avg       0.88      0.88      0.88     17357



## Second method

In [23]:
# Create TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1, 2))
x=tv.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(52071, 2500)
(17357, 2500)
(52071,)
(17357,)


In [25]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
from sklearn.metrics import *
print(f'accuracy_score:{accuracy_score(y_test,y_pred)}')
print(f'classification_report:{classification_report(y_test,y_pred)}')

accuracy_score:0.8738261220256957
classification_report:              precision    recall  f1-score   support

           0       0.91      0.79      0.85      3125
           1       0.89      0.91      0.90      5378
           2       0.89      0.86      0.87      4149
           3       0.83      0.90      0.86      4705

    accuracy                           0.87     17357
   macro avg       0.88      0.87      0.87     17357
weighted avg       0.88      0.87      0.87     17357

